In [1]:
import numpy as np

In [2]:
class Jaya():
    def __init__(self):
        self.obj = None
        self.population = None
        self.dimension = None
        self.ub = None
        self.lb = None
        self.optimization = "maximize"
        self.max_iter = None
        self.global_best_agent = None
        
    def compile(self,obj,pop_size,dimension,lb,ub,optimization="maximize"):
        self.obj = obj
        self.population_size = pop_size
        self.dimension = dimension
        self.ub = ub
        self.lb = lb
        self.optimization=optimization
        
    def __initialize(self):
        self.population = np.clip(np.random.rand(self.population_size,self.dimension),self.lb,self.ub)
        self.ev_value = np.empty(self.population_size)
        
        for i,value in enumerate(self.population):
            self.ev_value[i] = self.obj(value)
        
    def __update_best_worst(self):
        max_id = np.argmax(self.ev_value)
        min_id = np.argmin(self.ev_value)
        
        if self.optimization == "maximize":
            self.best_agent = (self.population[max_id],self.ev_value[max_id])
            self.worst_agent = (self.population[min_id],self.ev_value[min_id])
        elif self.optimization == "minimize":
            self.worst_agent = (self.population[max_id],self.ev_value[max_id])
            self.best_agent = (self.population[min_id],self.ev_value[min_id])
            
    def __update_global_best(self):
        if self.global_best_agent == None:
            self.global_best_agent = self.best_agent
        elif self.optimization == "maximize" and self.best_agent[1] > self.global_best_agent[1]:
            self.global_best_agent = self.best_agent
        elif self.optimization == "minimize" and self.best_agent[1] < self.global_best_agent[1]:
            self.global_best_agent = self.best_agent
           
    def train(self,max_iter):
        
        self.__initialize()
        self.__update_best_worst()
        for i in range(max_iter):
            
            self.__update_best_worst()
            self.__update_global_best()
            r1 = np.random.rand(self.population_size,self.dimension)
            r2 = np.random.rand(self.population_size,self.dimension)
            population_updated = self.population + r1 * (self.best_agent[0] - np.abs(self.population)) - r2 * (self.worst_agent[0] - np.abs(self.population))
            
            
            
            self.population = np.clip(population_updated,lb,ub)
            
            for i,value in enumerate(self.population):
                self.ev_value[i] = self.obj(value)     
        return self.global_best_agent
            

In [12]:
from benchmarking.benchmark import *
function=[Sphere,Schwefel_1_2,Rosenbrock,Ackley,SumSquare]


In [19]:
for i in function:
    func=i()
    values=np.empty(30)
    for l in range(30):
        lb = np.array([func.range[0]]).repeat(func.dim)
        ub = np.array([func.range[1]]).repeat(func.dim)
        jaya = Jaya()
        jaya.compile(func.get_algorithm(),pop_size=10,dimension=func.dim,lb=lb,ub=ub,optimization = "minimize")
        values[l]=jaya.train(100)[-1]
    print(f"{func.name}\t ==> Min = {np.min(values)} Max = {np.max(values)} Mean = {np.mean(values)}")

Sphere	 ==> Min = 5.307041577199469 Max = 9.564478360303445 Mean = 7.076345158824197
Schwefel_1_2	 ==> Min = 183.6309718496918 Max = 1388.4874913014648 Mean = 702.2592359841509
Rosenbrock	 ==> Min = 325.29673894195406 Max = 504.59488187977536 Mean = 400.76859878434306
Ackley	 ==> Min = 3.1154571495509065 Max = 3.735546495032786 Mean = 3.494417873139751
SumSquare	 ==> Min = 69.17509671347652 Max = 130.16389573409995 Mean = 101.11979661519159
